In [1]:
import re

In [2]:
def get_text(filepath):
    """
        get the text from input file
    """
    text = ""
    with open(filepath, "r", encoding ='utf-8') as inputfile:
        for line in inputfile:
            text = text + line
    return text


def find_text_begin_with_keyword(text, begin_keyword):
    found = re.search(begin_keyword, text)
    if found:
        return text[found.start():], found.start()
    else:
        return None


def find_last_text_begin_with_keyword(text, begin_keyword):
    begin = -1
    pivot = 0
    
    while True:
        found = find_text_begin_with_keyword(text[pivot:], begin_keyword)
        if found:
            begin = pivot + found[1]
            pivot = begin + len(begin_keyword)
        else:
            break
               
    if begin>=0:
        return text[begin:], begin
    else:
        return None


def find_last_text_begin_with_keyword_and_match_regex(text, begin_keyword, check_regex):
    begin = -1
    pivot = 0
    
    while True:
        found = find_text_begin_with_keyword(text[pivot:], begin_keyword)
        if found and re.match(check_regex, found[0]):
            begin = pivot + found[1]
            pivot = begin + len(begin_keyword)
        else:
            break
               
    if begin>=0:
        return text[begin:], begin
    else:
        return None


def cut_top_of_the_email(text):
    begin_keyword = "From:|Từ:"
    check_regex = r"(From:|Từ:).+[@].+(Date:|Ngày:)"
    data_found = find_last_text_begin_with_keyword_and_match_regex(text, begin_keyword, check_regex)
    if data_found:
        return data_found[0]
    else:
        return ""


def truncate_the_email(text):
    """
        get rid of the unnecessary text
    """
    result = text
    result = cut_top(result)
    return result


def extract_from_email_bodytext(inputfile):
    text = get_text(inputfile)
    text = truncate(text)
    
    return text

In [3]:
inputfile = "bodytext/1.txt"
print(extract_from_email_bodytext(inputfile))

NameError: name 'truncate' is not defined

In [ ]:
# def preprocess_and_split_datestring(datestring):
#     result = []
#     datestring = datestring.replace(",", "")
#     result = datestring.split()
#     return result


# def is_eng_date(datestring):
#     weekday_dict = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
#     return datestring[:3] in weekday_dict


# def is_vie_date(datestring):
#     pass


# def extract_eng_date(datestring):
#     """
#         get the day, month, year in the english datestring
#     """
        
#     month_dict = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', \
#                   'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}
#     tokens = preprocess_and_split_datestring(datestring)
    
#     year = ''
#     month = ''
#     day = ''
    
#     yearregex = r'(19|20)\d{2}'
#     dayregex = r'([12]\d|3[01]|0?[1-9])'
    
#     for token in tokens:
#         """
#             loop over all the integer in the string to check if it is day or month or year
#         """
#         if (month=='') and (token in month_dict):
#             month = month_dict[token]
#         elif (day=='') and re.match(dayregex, token):
#             day = token
#         elif (year=='') and re.match(yearregex, token):
#             year = token
            
#     if len(day)==1:
#         day = '0' + day
#     return day + '/' + month + '/' + year

In [ ]:
print(re.search("From", "abcFromabc").start())

In [26]:
ok = True
print(re.search("from", "abcFromabc", flags = (re.IGNORECASE if ok else 0)))

<re.Match object; span=(3, 7), match='From'>
